In [1]:
import numpy as np
import os
import glob

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
# I have multi-gpus

In [3]:
spectrogram = glob.glob('spectrogram-train/*npy')
len(spectrogram)

8184

In [4]:
def filter_text(string):
    string = string.lower()
    splitted = string.split('/')[1].split('.')[0].replace('<>','-').split('-')
    splitted = [w for w in splitted if not w.isdigit() and w not in ['man', 'woman', 'augment']]
    return ' '.join(splitted)

filter_text(spectrogram[-1])

'tolong sebut pariahship'

In [5]:
train_X, train_Y = [], []
for spec in spectrogram:
    train_Y.append(filter_text(spec))
    train_X.append(np.load(spec))

In [6]:
spectrogram = glob.glob('spectrogram-test/*npy')
len(spectrogram)

293

In [7]:
test_X, test_Y = [], []
for spec in spectrogram:
    test_Y.append(filter_text(spec))
    test_X.append(np.load(spec))

In [8]:
import tensorflow as tf
from tqdm import tqdm

/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: 

In [9]:
train_X = tf.keras.preprocessing.sequence.pad_sequences(
    train_X, dtype = 'float32', padding = 'post'
)

test_X = tf.keras.preprocessing.sequence.pad_sequences(
    test_X, dtype = 'float32', padding = 'post'
)

In [10]:
chars = list(set([c for target in train_Y + test_Y for c in target]))
num_classes = len(chars) + 2

idx2char = {idx + 1: char for idx, char in enumerate(chars)}
idx2char[0] = '<PAD>'
char2idx = {char: idx for idx, char in idx2char.items()}

In [11]:
train_Y = [[char2idx[c] for c in target] for target in train_Y]
test_Y = [[char2idx[c] for c in target] for target in test_Y]

In [12]:
def pad_sentence_batch(sentence_batch, pad_int):
    padded_seqs = []
    seq_lens = []
    max_sentence_len = max([len(sentence) for sentence in sentence_batch])
    for sentence in sentence_batch:
        padded_seqs.append(sentence + [pad_int] * (max_sentence_len - len(sentence)))
        seq_lens.append(len(sentence))
    return padded_seqs, seq_lens

def sparse_tuple_from(sequences, dtype=np.int32):
    indices = []
    values = []

    for n, seq in enumerate(sequences):
        indices.extend(zip([n] * len(seq), range(len(seq))))
        values.extend(seq)

    indices = np.asarray(indices, dtype=np.int64)
    values = np.asarray(values, dtype=dtype)
    shape = np.asarray([len(sequences), np.asarray(indices).max(0)[1] + 1], dtype=np.int64)

    return indices, values, shape

In [13]:
def pad_second_dim(x, desired_size):
    padding = tf.tile([[0]], tf.stack([tf.shape(x)[0], desired_size - tf.shape(x)[1]], 0))
    return tf.concat([x, padding], 1)

def position_encoding(inputs):
    T = tf.shape(inputs)[1]
    repr_dim = inputs.get_shape()[-1].value
    pos = tf.reshape(tf.range(0.0, tf.to_float(T), dtype=tf.float32), [-1, 1])
    i = np.arange(0, repr_dim, 2, np.float32)
    denom = np.reshape(np.power(10000.0, i / repr_dim), [1, -1])
    enc = tf.expand_dims(tf.concat([tf.sin(pos / denom), tf.cos(pos / denom)], 1), 0)
    return tf.tile(enc, [tf.shape(inputs)[0], 1, 1])

def layer_norm(inputs, epsilon=1e-8):
    mean, variance = tf.nn.moments(inputs, [-1], keep_dims=True)
    normalized = (inputs - mean) / (tf.sqrt(variance + epsilon))
    params_shape = inputs.get_shape()[-1:]
    gamma = tf.get_variable('gamma', params_shape, tf.float32, tf.ones_initializer())
    beta = tf.get_variable('beta', params_shape, tf.float32, tf.zeros_initializer())
    return gamma * normalized + beta

def cnn_block(x, dilation_rate, pad_sz, hidden_dim, kernel_size):
    x = layer_norm(x)
    pad = tf.zeros([tf.shape(x)[0], pad_sz, hidden_dim])
    x =  tf.layers.conv1d(inputs = tf.concat([pad, x, pad], 1),
                          filters = hidden_dim,
                          kernel_size = kernel_size,
                          dilation_rate = dilation_rate)
    x = x[:, :-pad_sz, :]
    x = tf.nn.relu(x)
    return x

class Model:
    def __init__(
        self,
        num_layers,
        size_layers,
        learning_rate,
        num_features,
        dropout = 1.0,
        kernel_size = 3,
        n_attn_heads = 16
    ):
        self.X = tf.placeholder(tf.float32, [None, None, num_features])
        self.Y = tf.sparse_placeholder(tf.int32)
        seq_lens = tf.count_nonzero(
            tf.reduce_sum(self.X, -1), 1, dtype = tf.int32
        ) + 10
        filled = tf.fill(tf.shape(seq_lens), tf.shape(self.X)[1])
        seq_lens = tf.where(seq_lens > tf.shape(self.X)[1], filled, seq_lens)
        self.label = tf.placeholder(tf.int32, [None, None])
        self.Y_seq_len = tf.placeholder(tf.int32, [None])
        forward = tf.layers.conv1d(self.X, size_layers, 
                                   kernel_size = 1, strides = 1, padding = 'SAME')
        encoder_embedded = tf.identity(forward)
        for i in range(num_layers): 
                    dilation_rate = 2 ** i
                    pad_sz = (kernel_size - 1) * dilation_rate 
                    with tf.variable_scope('block_%d'%i):
                        encoder_embedded += cnn_block(encoder_embedded, dilation_rate, 
                                       pad_sz, size_layers, kernel_size)
        
        encoder_embedded = tf.nn.sigmoid(encoder_embedded)
        logits = tf.layers.dense(encoder_embedded, num_classes)
        time_major = tf.transpose(logits, [1, 0, 2])
        decoded, log_prob = tf.nn.ctc_beam_search_decoder(time_major, seq_lens)
        decoded = tf.to_int32(decoded[0])
        self.preds = tf.sparse_tensor_to_dense(decoded)
        self.cost = tf.reduce_mean(
            tf.nn.ctc_loss(
                self.Y,
                time_major,
                seq_lens
            )
        )
        self.optimizer = tf.train.AdamOptimizer(
            learning_rate = learning_rate
        ).minimize(self.cost)
        
        preds = self.preds[:, :tf.reduce_max(self.Y_seq_len)]
        masks = tf.sequence_mask(self.Y_seq_len, tf.reduce_max(self.Y_seq_len), dtype=tf.float32)
        preds = pad_second_dim(preds, tf.reduce_max(self.Y_seq_len))
        y_t = tf.cast(preds, tf.int32)
        self.prediction = tf.boolean_mask(y_t, masks)
        mask_label = tf.boolean_mask(self.label, masks)
        self.mask_label = mask_label
        correct_pred = tf.equal(self.prediction, mask_label)
        correct_index = tf.cast(correct_pred, tf.float32)
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [14]:
tf.reset_default_graph()
sess = tf.InteractiveSession()

size_layers = 256
learning_rate = 1e-3
num_layers = 3
batch_size = 128
epoch = 20

model = Model(num_layers, size_layers, learning_rate, train_X[0].shape[1])
sess.run(tf.global_variables_initializer())

W0818 11:22:52.214461 140542686246720 deprecation.py:506] From /home/husein/.local/lib/python3.6/site-packages/tensorflow/python/util/deprecation.py:507: calling count_nonzero (from tensorflow.python.ops.math_ops) with axis is deprecated and will be removed in a future version.
Instructions for updating:
reduction_indices is deprecated, use axis instead
W0818 11:22:52.233758 140542686246720 deprecation.py:323] From <ipython-input-13-2e47c29a6de4>:50: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0818 11:22:52.237230 140542686246720 deprecation.py:323] From <ipython-input-13-2e47c29a6de4>:54: conv1d (from tensorflow.python.layers.convolutional) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.keras.layers.Conv1D` instead.
W0818 11:22:52.241859 140542686246720 deprecati

In [15]:
for e in range(epoch):
    pbar = tqdm(
        range(0, len(train_X), batch_size), desc = 'minibatch loop')
    train_cost, train_accuracy, test_cost, test_accuracy = [], [], [], []
    for i in pbar:
        batch_x = train_X[i : min(i + batch_size, len(train_X))]
        y = train_Y[i : min(i + batch_size, len(train_X))]
        batch_y = sparse_tuple_from(y)
        batch_label, batch_len = pad_sentence_batch(y, 0)
        _, cost, accuracy = sess.run(
            [model.optimizer, model.cost, model.accuracy],
            feed_dict = {model.X: batch_x, model.Y: batch_y, 
                         model.label: batch_label, model.Y_seq_len: batch_len},
        )
        train_cost.append(cost)
        train_accuracy.append(accuracy)
        pbar.set_postfix(cost = cost, accuracy = accuracy)
    
    pbar = tqdm(
        range(0, len(test_X), batch_size), desc = 'testing minibatch loop')
    for i in pbar:
        batch_x = test_X[i : min(i + batch_size, len(test_X))]
        y = test_Y[i : min(i + batch_size, len(test_X))]
        batch_y = sparse_tuple_from(y)
        batch_label, batch_len = pad_sentence_batch(y, 0)
        cost, accuracy = sess.run(
            [model.cost, model.accuracy],
            feed_dict = {model.X: batch_x, model.Y: batch_y, 
                         model.label: batch_label, model.Y_seq_len: batch_len},
        )
        
        test_cost.append(cost)
        test_accuracy.append(accuracy)
        
        pbar.set_postfix(cost = cost, accuracy = accuracy)
    
    print('epoch %d, training avg cost %f, training avg accuracy %f'%(e + 1, np.mean(train_cost), 
                                                                      np.mean(train_accuracy)))
    
    print('epoch %d, testing avg cost %f, testing avg accuracy %f'%(e + 1, np.mean(test_cost), 
                                                                    np.mean(test_accuracy)))

minibatch loop:   0%|          | 0/64 [00:00<?, ?it/s]

epoch 1, training avg cost 66.266273, training avg accuracy 0.072635
epoch 1, testing avg cost 51.142719, testing avg accuracy 0.129454


minibatch loop:   0%|          | 0/64 [00:00<?, ?it/s]

epoch 2, training avg cost 43.313988, training avg accuracy 0.262416
epoch 2, testing avg cost 35.697575, testing avg accuracy 0.354671


minibatch loop:   0%|          | 0/64 [00:00<?, ?it/s]

...

epoch 18, training avg cost 10.745575, training avg accuracy 0.597572
epoch 18, testing avg cost 22.457262, testing avg accuracy 0.585130


minibatch loop:   0%|          | 0/64 [00:00<?, ?it/s]

epoch 19, training avg cost 10.153742, training avg accuracy 0.601557
epoch 19, testing avg cost 22.512693, testing avg accuracy 0.585935


testing minibatch loop: 100%|██████████| 3/3 [00:05<00:00,  1.72s/it, accuracy=0.606, cost=22.8]

epoch 20, training avg cost 9.581175, training avg accuracy 0.605466
epoch 20, testing avg cost 22.805923, testing avg accuracy 0.593097


In [17]:
import random

random_index = random.randint(0, len(test_X) - 1)
batch_x = test_X[random_index : random_index + 1]
print(
    'real:',
    ''.join(
        [idx2char[no] for no in test_Y[random_index : random_index + 1][0]]
    ),
)
batch_y = sparse_tuple_from(test_Y[random_index : random_index + 1])
pred = sess.run(model.preds, feed_dict = {model.X: batch_x})[0]
print('predicted:', ''.join([idx2char[no] for no in pred]))

real: sebut perkataan ambasador
predicted: sebut perkatan ambaton
